#**Ch03. 평가**
* 머신러닝은 데이터 가공/변환, 모델 학습/예측, 그리고 평가의 프로세스로 구성됨
* 예측 성능을 평가하는 방법 : 성능평가지표(Evaluation Metric)
  * 회귀 : 실제값과 예측값의 오차 평균값에 기반
  * 분류 : 실제 결과 데이터와 예측 겨로가 데이터가 얼마나 정확하고 오류가 적게 발생하는가에 기반


* 분류의 성능 평가 지표(이진/멀티, 이진에서 더욱 강조하는 지표)
  * 정확도(Accuracy)
  * 오차 행렬(Confusion Matrix)
  * 정밀도(Precision)
  * 재현율 (Recall)
  * F1 score
  * ROC AUC



##**01. 정확도(Accuracy)**
- 정확도 = 예측 결과가 동일한 데이터 건수/ 전체 예측 데이터 건수
- 정확도 지표가 어떻게 모델의 성능을 왜곡하는가?


---


사이킷런의 BaseEstimator 클래스를 활용하여, 단순히 성별에 따라 생존자를 예측하는 단순한 분류기를 생성

- 사이킷런의 BaseEsimators를 활용하면 Customized된 Estimator를 생성할 수 있음

In [30]:
from sklearn.base import BaseEstimator

class MyDummyClassifier(BaseEstimator):
  #fit()메서드는 아무것도 학습하지 않음
  def fit(self, X, y=None):
    pass
  #predict() 메서드는 단순히 Sex 피처가 1이면 0, 그렇지 않으면 1로 예측함
  def predict(self, X):
    pred = np.zeros((X.shape[0],1))
    for i in range(X.shape[0]):
      if X['Sex'].iloc[i]==1:
        pred[i]=0
      else:
        pred[i]=1
    
    return pred

In [31]:
#MyDummyClassifier을 이용해 앞 장의 타이타닉 생존자 예측을 수행
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import numpy as np
#Null 처리 함수
def fillna(df):
  df['Age'].fillna(df['Age'].mean(),inplace=True)
  df['Cabin'].fillna('N',inplace=True)
  df['Embarked'].fillna('N',inplace=True)
  df['Fare'].fillna(0, inplace=True)
  return df

#머신러닝 알고리즘에 불필요한 속성제거
def drop_features(df):
  df.drop(['PassengerId','Name','Ticket'],axis=1, inplace=True)
  return df

#레이블 인코딩 수행
def format_features(df):
  df['Cabin'] = df['Cabin'].str[:1]
  features = ['Cabin','Sex','Embarked']
  for feature in features:
    le = preprocessing.LabelEncoder()
    le = le.fit(df[feature])
    df[feature] = le.transform(df[feature])
  return df

#앞에서 설정한 데이터 전처리 함수 호출
def transform_features(df):
  df = fillna(df)
  df = drop_features(df)
  df = format_features(df)
  return df

In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 원본 데이터를 재로딩, 데이터 가공, 학습 데이터/테스트 데이터 분할
titanic_df=pd.read_csv('/content/drive/MyDrive/ESAA/22-2/DATA/titanic_train.csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived',axis=1)
X_titanic_df = transform_features(X_titanic_df)
X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, test_size=0.2, random_state=0)

# 위에서 생성한 Dummy Classifier를 이용해 학습/예측/평가 수행
myclf=MyDummyClassifier()
myclf.fit(X_train,y_train)

mypredictions = myclf.predict(X_test)
print('Dummy Classifier의 정확도는: {0:.4f}'.format(accuracy_score(y_test,mypredictions)))

Dummy Classifier의 정확도는: 0.7877


-> 단순 알고리즘으로 예측하는 경우에도 데이터의 구성에 따라 정확도 결과는 78.77%로 매우 높게 나올 수 있음


---

- MNIST 데이터 세트를 변환하여 불균형한 데이터 세트를 만든 뒤 정확도 지표 적용시 어떤 문제가 발생할 수 있는지 살펴보기

- **MNIST 데이터세트** : 0부터 9까지의 숫자 이미지의 픽셀 정보를 가지고 있으며, 이를 기반으로 숫자 Digit을 예측하는데 사용
  - 0부터 9까지의 멀티레이블이지만 레이블 값이 7인 것만 True, 나머지는 False인 불균형한 데이터 세트로 변환
  - 이후에 모든 데이터를 False(0)으로 예측하는 분류기를 만들어 정확도를 측정
결과적으로 아무것도 하지 않고, 특정 결과로만 결과를 반환해도 정확도가 높게 측정되어 모델 성능이 높게 나타나는 현상이 발생

In [35]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

class MyFakeClassifier(BaseEstimator):
  def fit(self, X, y):
    pass

  # 입력값으로 들어오는 X 데이터 세트의 크기만큼 모두 0값으로 만들어서 반환
  def predict(self, X):
    return np.zeros((len(X),1),dtype=bool)

# 사이킷런의 내장 데이터 셋인 load_digits()를 이용하여 MNIST 데이터 로딩
digits = load_digits()

# digits 번호가 7이면 True이고 이를 astype(int)로 1로 변환, 7이 아니면 False이고 0으로 변환
y=(digits.target == 7).astype(int)

# 훈련셋, 테스트셋으로 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(digits.data, y, random_state=11)

In [36]:
# 불균형한 레이블 데이터 분포도 확인
print('레이블 테스트 데이터 크기: ', y_test.shape)
print('테스트 데이터 세트 레이블 0과 1의 분포도: ')
print(pd.Series(y_test).value_counts())

# FakeClassifier를 통해 학습/예측/정확도 평가
fakeclf = MyFakeClassifier()
fakeclf.fit(X_train, y_train)
fake_prediction = fakeclf.predict(X_test)
print('모든 예측을 0으로 했을 때의 정확도: ', accuracy_score(y_test, fake_prediction))

레이블 테스트 데이터 크기:  (450,)
테스트 데이터 세트 레이블 0과 1의 분포도: 
0    405
1     45
dtype: int64
모든 예측을 0으로 했을 때의 정확도:  0.9


##**02. 오차 행렬(Confusion matrix)**

예측을 수행하면서 얼마나 헷갈리고 있는지도 함께 보여주는 지표

**오차행렬**
: 4분면으로 나누어 왼쪽, 오른쪽을 예측된 클래스 값 기준으로 Negative와 Positive로 분류, 위 아래를 실제 클래스 값 기준으로 Negative와 Positive로 분류

- TN: True Negative
- FP: False Positive
- FN: False Negative
- TP: True Positive

In [ ]:
#사이킷런의 오차행렬 API: confusion_matrix()
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test,fakepred)

**정확도 = 예측 결과와 실제 값이 동일한 건수/전체 데이터 수 = (TN + TP)/(TN + FP + FN + TP)**

Positive 양성으로 1, Negatie 음성으로 0 값을 각각 할당

불균형한 이진 분류 데이터 세트에서는 Positice 데이터 건수가 매우 작기 때문에 Negative로 예측 정확도가 높아지는 경향이 발생

## **03. 정밀도와 재현율**

Positive 데이터 세트의 예측 성능에 좀 더 초점을 맞춤

**정밀도 = TP / (FP + TP)**

**재현율 = TP / (FN + TP)**

정밀도는 예측을 Positive로 한 대상 중 예측과 실제 값이 Positive로 일치한 데이터 비율

재현율은 실제 값이 Positive인 대상 중에 예측과 실제 값이 Positive로 일치한 데이터 비율

In [ ]:
#오차 행렬 및 정밀도, 재현율을 모두 구해서 예측 성능을 평가
#사이킷런의 정밀도 계산: precision_score()
#재현율 계산: recall_score()
#평가 간편 적용:get_clf_eval()

from sklearn.metrics import accuracy_score,precision_score,precision_score,recall_score,confusion_matrix

def get_clf_eval(y_test,pred):
  confusion=confusion_matrix(y_test,pred)
  accuracy=accuracy_score(y_test,pred)
  precision=precision_score(y_test,pred)
  recall=recall_score(y_test,pred)
  print('오차 행렬')
  print(confusion)
  print('정확도:{0:.4f},정밀도:{1:.4f},재현율:{2:.4f}'.format(accuracy,precision,recall))

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

#원본 데이터를 재로딩, 데이터 가공, 학습 데이터/테스트 데이터 분할
titanic_df=pd.read_csv('./titanic_train.csv')
y_titanic_df=titanic_df['Survived']
X_titanic_df=titanic_df.drop('Survived',axis=1)
X_titanic_df=transform_features(X_titanic_df)

X_train,X_test,y_train,y_test=train_test_split(X_titanic_df,y_titanic_df,test_size=0.20,random_state=11)

lr_clf=LogisticRegression()

lr_clf.fit(X_train,y_train)
pred=lr_clf.predict(X_test)
get_clf_eval(y_test,pred)

#### **정밀도/재현율 트레이드오프**

정밀도와 재현율은 상호 보완적인 평가 지표이므로 한쪽을 강제로 높이면 다른 하나는 수치가 떨어짐

사이킷런의 분류 알고리즘은 각각 레이블별 결정 확률을 구하고 더 큰 확률을 가진 것을 예측값으로 함

**predict_proba( )**: 예측 확률을 반환

In [ ]:
pred_proba=lr_clf.predict_proba(X_test)
pred=lr_clf.predict(X_test)
print('pred_proba( ) 의 결과 Shape: {0}'.format(pred_proba.shape))
print('pred_proba array에서 앞 3개만 샘플로 추출 \n:', pred_proba[:3])

pred_proba_result=np.concatenate([pred_proba, pred.reshape(-1, 1)], axis=1)
print('두 개의 class 중에서 더 큰 확률을 클래스 값으로 예측 \n', pred_proba_result[:3])

In [ ]:
from sklearn.preprocessing import Binarizer

X=[[1, -1, 2],
   [2, 0, 0],
   [0, 1.1, 1.2]]

binarizer=Binarizer(threshold=1.1)
print(binarizer.fit_transform(X))
#1.1보다 크면 1을 갖도록

In [ ]:
from sklearn.preprocessing import Binarizer

custom_threshold=0.5

pred_proba_1=pred_proba[:,1].reshape(-1,1)

binarizer=Binarizer(threshold=custom_threshold).fit(pred_proba_1)
custom_predict=binarizer.transform(pred_proba_1)

get_clf_eval(y_test, custom_predict)

In [ ]:
#임곗값을 0.4로
custom_threshold=0.4

pred_proba_1=pred_proba[:,1].reshape(-1,1)

binarizer=Binarizer(threshold=custom_threshold).fit(pred_proba_1)
custom_predict=binarizer.transform(pred_proba_1)

get_clf_eval(y_test, custom_predict)

In [ ]:
thresholds=[0.4, 0.45, 0.5, 0.55, 0.6]

In [ ]:
def get_eval_by_threshold(y_test, pred_proba_c1, thresholds):
  for custom_threshold in thresholds:
    binarizer=Binarizer(threshold=custom_threshold).fit(pred_proba_c1)
    custom_predict=binarizer.transform(pred_proba_c1)
    print('임곗값:', custom_threshold)
    get_clf_eval(y_test,custom_predict)

get_eval_by_threshold(y_test, pred_proba[:,1].reshape(-1,1), thresholds)

In [ ]:
from sklearn.metrics import precision_recall_curve

#레이블 값이 1일 때의 예측 확률을 추출
pred_proba_class1=lr_clf.predict_proba(X_test)[:,1]

#실제값 데이터 세트와 레이블 값이 1일 때의 예측 확률을 precision_recall_curve 인자로 입력
precisions, recalls, thresholds = precision_recall_curve(y_test, pred_proba_class1)
print('반환된 분류 결정 임곗값 배열의 Shape:', thresholds.shape)

#반환된 임계값 배열 로우가 147건이므로 샘플로 10건만 추출하되, 임계값을 15 Step으로 추출
thr_index=np.arange(0, thresholds.shape[0], 15)
print('샘플 추출을 위한 임계값 배열의 index 10개:', thr_index)
print('샘플용 10개의 임곗값:', np.round(thresholds[thr_index],2))

#15step 단위로 추출된 임계값에 따른 정밀도와 재현율 값
print('샘플 임계값별 정밀도:', np.round(precisions[thr_index],3))
print('샘플 임계값별 재현율:', np.round(recalls[thr_index],3))


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline

In [ ]:
def precision_recall_curve_plot(y_test, pred_proba_c1):
  precisions, recalls, thresholds = precision_recall_curve(y_test, pred_proba_c1)

  plt.figure(figsize=(8,6))
  threshold_boundary=thresholds.shape[0]
  plt.plot(thresholds, precisions[0:threshold_boundary], linestyle='-', label='precsion')
  plt.plot(thresholds, recalls[0:threshold_boundary], label='recall')

  start, end=plt.xlim()
  plt.xticks(np.round(np.arange(start, end, 0.1),2))

  plt.xlabel('Threshold value'); plt.ylabel('Precision and Recall value')
  plt.legend( ); plt.grid()
  plt.show()

precision_recall_curve_plot(y_test, lr_clf.predict_proba(X_test)[:,1])

#### **정밀도와 재현율의 맹점**

임곗값을 변경함에 따라 정밀도와 재현율의 수치가 변경됨
-> 그러므로 두 개의 수치를 상호 보완할 수 있는 수준에서 적용

- 정밀도 100%가 되는 방법

:확실한 기준이 되는 경우만 Positice로 에측하고 나머지는 모두 Negative로 예측

FP=0, TP=1

- 재현율이 100%가 되는 방법

: 모든 환자를 Positive로 예측

TP=30, FN=0

## **04. F1 스코어**

- 정밀도와 재현율을 결합한 지표
- 정밀도와 재현율이 어느 한 쪽으로 치우치지 않는 수치를 나타낼 때 상대적으로 높은 값을 가짐

In [ ]:
from sklearn.metrics import f1_score
f1=f1_score(y_test, pred)
print('F1 스코어: {0:.4f}'.format(f1))

In [ ]:
#get_clf_eval() 함수에 F1 스코어 추가

def get_clf_eval(y_test, pred):
  confusion=confusion_matrix(y_test, pred)
  accuracy=accuracy_score(y_test, pred)
  precision=precision_score(y_test, pred)
  recall=recall_score(y_test, pred)

  f1=f1_score(y_test, pred)
  print('오차 행렬')
  print(confusion)

  print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1: {3:.4f}'.format(accuracy, precision, recall, f1))

thresholds=[0.4, 0.45, 0.5, 0.55, 0.6]
pred_proba=lr_clf.predict_proba(X_test)
get_eval_by_threshold(y_test, pred_proba[:,1].reshape(-1,1), thresholds)

## **05. ROC 곡선과 AUC**

- ROC 곡선: 수신자 판단 곡선, FPR을 X축, TPR을 Y축으로
- TPR=True Positive Rate=재현율, 민감도=**TP/(FN + TP)**
- TNR=True Negative Rate=특이성=**TN/(FP + TN)**
- FPR=False Positive Rate= 1- 특이성=**FP/(FP + TN)**

---

ROC 곡선은 대각선으로 이은 직선과 멀수록 성능이 좋고 가까울수록 성능이 안좋음

FPR을 변경하면서 TPR의 변화 값을 구함

FPR=0 : 임곗값이 1 = positive로 예측하는 경우가 없으므로 FP=0

FPR=1 : 임곗값이 0 = 전부 positive로 예측하므로 TN=0이 됨

---
### **roc_curve( )**:

- 입력 파라미터: y_true: 실제 클래스 값 arrary, y_score: predict_prova의 반환 값
- 반환값: fpr, thresholds


In [ ]:
from sklearn.metrics import roc_curve
pred_proba_class1=lr_clf.predict_proba(X_test)[:,1]

fprs, tprs, thresholds=roc_curve(y_test, pred_proba_class1)

thr_index=np.arange(0, thresholds.shape[0],5)
#결과값이 똑같이 나오려면 0부터 시작해야하는데 왜 교재에는 1부터 시작하는가?..

print('샘플 추출을 위한 임곗값 배열의 index 10개:', thr_index)
print('샘플용 10개의 임곗값: ',np.round(thresholds[thr_index],2))

print('샘플 임곗값별 FPR: ', np.round(fprs[thr_index],3))
print('샘플 임곗값별 TPR: ', np.round(tprs[thr_index],3))

In [ ]:
def roc_curve_plot(y_test, pred_proba_c1):
  # 임곘값에 따른 FPR, TPR 값을 반환받음
  fprs, tprs, thresholds=roc_curve(y_test, pred_proba_c1)
  # ROC 곡선을 그래프 곡선으로 그림
  plt.plot(fprs, tprs, label='ROC')
  # 가운데 대각선 직선을 그림
  plt.plot([0,1],[0,1], 'k--', label='Random')

  # FPR X 축의 Scale을 0.1 단위로 변경, X, Y축 명 설정 등
  start, end = plt.xlim()
  plt.xticks(np.round(np.arange(start, end, 0.1), 2))
  plt.xlim(0,1); plt.ylim(0,1)
  plt.xlabel('FPR( 1 - Sensitivity)'); plt.ylabel('TPR( Recall )')
  plt.legend()

roc_curve_plot(y_test, pred_proba[:,1])

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
from sklearn.metrics import recall_score, f1_score, roc_auc_score
import numpy as np

#기존의 교재에 있는 y_target, preds가 뭔지 모르겠습니다.

print(confusion_matrix(y_test, pred))
print('정확도: ', np.round(accuracy_score(y_test, pred), 4))
print('정밀도: ', np.round(precision_score(y_test, pred), 4))
print('재현율: ', np.round(recall_score(y_test, pred), 4))

In [ ]:
#get_clf_eval에 ROC AUC 값추가

def get_clf_eval(y_test, pred):
  confusion=confusion_matrix(y_test, pred)
  accuracy=accuracy_score(y_test, pred)
  precision=precision_score(y_test, pred)
  recall=recall_score(y_test, pred)
  f1=f1_score(y_test, pred)

  roc_auc=roc_auc_score(y_test, pred_proba)
  print('오차 행렬')
  print(confusion)

  print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1: {3:.4f}'.format(accuracy, precision, recall, f1, roc_auc))
